In [ ]:
# Downgrade NumPy to version 1.x (e.g. 1.24.4)
!pip install numpy==1.24.4


In [1]:
!pip install scikit-surprise


In [2]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip


--2025-07-18 10:48:06--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip.1’

ml-100k.zip.1       100%[===================>]   4.70M  15.9MB/s    in 0.3s    

2025-07-18 10:48:07 (15.9 MB/s) - ‘ml-100k.zip.1’ saved [4924029/4924029]

Archive:  ml-100k.zip
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflat

In [3]:
import pandas as pd
from surprise import Dataset, Reader

# Load u.data into pandas DataFrame
df = pd.read_csv('ml-100k/u.data', sep='\t', names=['userID', 'itemID', 'rating', 'timestamp'])

# Use Reader to load from DataFrame
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)


In [4]:
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
model = SVD()
model.fit(trainset)
predictions = model.test(testset)

rmse(predictions)


RMSE: 0.9375


0.9374763904174391

In [5]:
from collections import defaultdict

def get_top_n(predictions, n=5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

top_n = get_top_n(predictions, n=5)

# Display for 5 users
for uid, user_ratings in list(top_n.items())[:5]:
    print(f"\nUser {uid} recommended items:")
    for (iid, rating) in user_ratings:
        print(f"  Movie {iid} with predicted rating {rating:.2f}")



User 907 recommended items:
  Movie 143 with predicted rating 5.00
  Movie 98 with predicted rating 5.00
  Movie 173 with predicted rating 4.98
  Movie 647 with predicted rating 4.91
  Movie 1 with predicted rating 4.90

User 371 recommended items:
  Movie 210 with predicted rating 4.28
  Movie 186 with predicted rating 4.14
  Movie 746 with predicted rating 4.10
  Movie 237 with predicted rating 4.05
  Movie 175 with predicted rating 4.03

User 218 recommended items:
  Movie 12 with predicted rating 4.07
  Movie 654 with predicted rating 4.02
  Movie 209 with predicted rating 3.76
  Movie 42 with predicted rating 3.46
  Movie 47 with predicted rating 3.43

User 829 recommended items:
  Movie 408 with predicted rating 4.18
  Movie 213 with predicted rating 4.03
  Movie 198 with predicted rating 3.97
  Movie 190 with predicted rating 3.86
  Movie 170 with predicted rating 3.85

User 733 recommended items:
  Movie 515 with predicted rating 3.92
  Movie 14 with predicted rating 3.60
  Mo